A simple example of starting a dask cluster with `mpirun` and having each worker in the cluster create an array, pass that array down to MPI, and scattering the arrays to each other.

In order to run this notebook, you will need to run a dask-scheduler with the following command:
`dask-scheduler --scheduler-file cluster.json`

And run 4 workers inside of mpirun with the following command:
`mpirun -np 4 dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json`

In [ ]:
from dask.distributed import Client, wait
import numpy as np

*Note:* It's important that the workers in your dask-mpi cluster have been started with `nthreads [>1]`. Otherwise the collective communications will deadlock.

Change the `scheduler_file` argument below to point to your scheduler file.

In [3]:
client = Client(scheduler_file = "/home/cjnolet/cluster.json", direct_to_workers = True)

In [5]:
client

Client Scheduler: tcp://10.2.171.62:8786 Dashboard: http://10.2.171.62:8787/status,Cluster Workers: 4 Cores: 40 Memory: 62.99 GB


In [6]:
def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

In [7]:
workers = [parse_host_port(worker) for worker in list(client.has_what().keys())]

In [8]:
from mpi4py import MPI

def get_ranks(l):
    try:
        comm = MPI.COMM_WORLD
        rank = comm.Get_rank()
        return rank
    
    except Exception as e:
        print(str(e))

In [9]:
def get_size():
    comm = MPI.COMM_WORLD
    return comm.Get_size()

In [10]:
# How many workers recognized in the MPI world?
client.submit(get_size).result()

4

### Get worker ranks

In [11]:
# Sending a unique value to each worker to make sure the function gets propagated to all the workers
pmpi = [(client.submit(get_ranks, ident, workers = [worker]), worker) for worker, ident in zip(workers, range(len(workers)))]
pmpi

[(<Future: status: pending, key: get_ranks-c1c698d7265b3884daf93d8830905938>,
  ('10.2.171.62', 45457)),
 (<Future: status: pending, key: get_ranks-8696fb4f9cd0ee5192b71c0e474898e4>,
  ('10.2.171.62', 42106)),
 (<Future: status: pending, key: get_ranks-be89f0b062970fce534df6cb95cbcfa0>,
  ('10.2.171.62', 37576)),
 (<Future: status: pending, key: get_ranks-569e64d811f8ef6822088996082d3ea1>,
  ('10.2.171.62', 34855))]

In [12]:
ranks = dict([(r.result(), worker) for r, worker in pmpi])

In [13]:
ranks

{0: ('10.2.171.62', 42106),
 1: ('10.2.171.62', 34855),
 2: ('10.2.171.62', 45457),
 3: ('10.2.171.62', 37576)}

First let's push some data to the workers for them to send to each other.

In [14]:
def create_np(rank):
    return np.arange(rank, rank+50)

In [15]:
np.arange(0, 50).dtype

dtype('int64')

In [16]:
res = [client.submit(create_np, rank, workers = [ranks[rank]]) for rank in ranks]
wait(res)

DoneAndNotDoneFutures(done={<Future: status: finished, type: ndarray, key: create_np-2fb2c90159778f8dde184a06e502f581>, <Future: status: finished, type: ndarray, key: create_np-918cf2366aa5fa053ef38b999d0cbac8>, <Future: status: finished, type: ndarray, key: create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5>, <Future: status: finished, type: ndarray, key: create_np-46fbb7eeb4cad20b72102f33b0968993>}, not_done=set())

In [17]:
# Verify workers have generated data
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',)}

### Test point-to-point
An initial test to prove that we can do point-to-point communications across workers

In [18]:
list(range(1, 4))

[1, 2, 3]

In [19]:
def test_point_to_point(d):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    if rank == 0:
        data = d
        for i in range(1, 4):
            print("Sending to %s" % i)
            req = comm.isend(data, dest=i, tag=11)
        req.wait()
        return "sender"
    else:
        req = comm.irecv(source=0, tag=11)
        data = req.wait()
        return data

In [20]:
sender = [client.submit(test_point_to_point, future) for future in res]
sender

[<Future: status: pending, key: test_point_to_point-1cb97e86c6e1dea824ab997523915c2c>,
 <Future: status: pending, key: test_point_to_point-6b280794ec9c352f0c5a56594041eb78>,
 <Future: status: pending, key: test_point_to_point-b4e50abc7782250bf416c4901783d9af>,
 <Future: status: pending, key: test_point_to_point-c06f983984d95bda257d3cb38e6006a1>]

In [21]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-1cb97e86c6e1dea824ab997523915c2c': ('tcp://10.2.171.62:42106',),
 'test_point_to_point-6b280794ec9c352f0c5a56594041eb78': ('tcp://10.2.171.62:34855',),
 'test_point_to_point-b4e50abc7782250bf416c4901783d9af': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-c06f983984d95bda257d3cb38e6006a1': ('tcp://10.2.171.62:37576',)}

In [22]:
client.gather(sender)

['sender',
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])]

### Test point-to-point with Numpy arrays 

In [23]:
def test_point_to_point_np(data):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    # automatic MPI datatype discovery
    if rank == 0:
        for i in range(1, 4):
            comm.Send(data, dest=i, tag=13)
        return data
    else:
        comm.Recv(data, source=0, tag=13)
        return data

In [24]:
nodes = [client.submit(test_point_to_point_np, future) for future in res]
nodes

[<Future: status: pending, key: test_point_to_point_np-ce0113c4a9a80bc9fcd320e18c8b8524>,
 <Future: status: pending, key: test_point_to_point_np-cac317307bda26b7dea8b332919f3710>,
 <Future: status: pending, key: test_point_to_point_np-b1aef79edfd00cfb63760bc37f2cbfd7>,
 <Future: status: pending, key: test_point_to_point_np-3ec074276748f07bc003ae97765a0b0c>]

In [25]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-1cb97e86c6e1dea824ab997523915c2c': ('tcp://10.2.171.62:42106',),
 'test_point_to_point-6b280794ec9c352f0c5a56594041eb78': ('tcp://10.2.171.62:34855',),
 'test_point_to_point-b4e50abc7782250bf416c4901783d9af': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-c06f983984d95bda257d3cb38e6006a1': ('tcp://10.2.171.62:37576',),
 'test_point_to_point_np-3ec07427674

In [26]:
client.gather(nodes)

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])]

In [27]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-1cb97e86c6e1dea824ab997523915c2c': ('tcp://10.2.171.62:42106',),
 'test_point_to_point-6b280794ec9c352f0c5a56594041eb78': ('tcp://10.2.171.62:34855',),
 'test_point_to_point-b4e50abc7782250bf416c4901783d9af': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-c06f983984d95bda257d3cb38e6006a1': ('tcp://10.2.171.62:37576',),
 'test_point_to_point_np-3ec07427674

### Test Bcast

In [28]:
def test_broadcast(d):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    
    try:
        if rank == 0:
            data = d
        else:
            data = None
        data = comm.bcast(data, root=0)
        return (rank, data)
    except Exception as e:
        print(str(e))

In [29]:
nodes = [client.submit(test_broadcast, future) for future in res]
nodes

[<Future: status: pending, key: test_broadcast-1cbbffb737592187644be238ee1f9682>,
 <Future: status: pending, key: test_broadcast-75274ada5e6cad21e25e663bf5f43f73>,
 <Future: status: pending, key: test_broadcast-bc708cdf85134882200b5f61b831f396>,
 <Future: status: pending, key: test_broadcast-72a1a64a7cc2cc322af24bb9d307b35d>]

In [30]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_broadcast-1cbbffb737592187644be238ee1f9682': ('tcp://10.2.171.62:42106',),
 'test_broadcast-72a1a64a7cc2cc322af24bb9d307b35d': ('tcp://10.2.171.62:37576',),
 'test_broadcast-75274ada5e6cad21e25e663bf5f43f73': ('tcp://10.2.171.62:34855',),
 'test_broadcast-bc708cdf85134882200b5f61b831f396': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-1cb97e86c6e1dea824ab997523915c2c':

In [31]:
nodes

[<Future: status: finished, type: tuple, key: test_broadcast-1cbbffb737592187644be238ee1f9682>,
 <Future: status: finished, type: tuple, key: test_broadcast-75274ada5e6cad21e25e663bf5f43f73>,
 <Future: status: finished, type: tuple, key: test_broadcast-bc708cdf85134882200b5f61b831f396>,
 <Future: status: finished, type: tuple, key: test_broadcast-72a1a64a7cc2cc322af24bb9d307b35d>]

In [32]:
client.gather(nodes)

[(0, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])),
 (1, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])),
 (2, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])),
 (3, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]))]

### Test Scatter

In [33]:
def test_scatter(r):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()

    sendbuf = None
    if rank == 0:
        sendbuf = np.empty([size, 100], dtype='i')
        sendbuf.T[:,:] = range(size)
    recvbuf = np.empty(100, dtype='i')
    comm.Scatter(sendbuf, recvbuf, root=0)
    return np.allclose(recvbuf, rank)

In [34]:
client.has_what(ranks[0][0] +":" + str(ranks[0][1]))

{'tcp://10.2.171.62:42106': ('get_ranks-8696fb4f9cd0ee5192b71c0e474898e4',
  'test_broadcast-1cbbffb737592187644be238ee1f9682',
  'test_point_to_point_np-ce0113c4a9a80bc9fcd320e18c8b8524',
  'create_np-918cf2366aa5fa053ef38b999d0cbac8',
  'test_point_to_point-1cb97e86c6e1dea824ab997523915c2c')}

In [35]:
nodes = [client.submit(test_scatter, future) for future in res]
nodes

[<Future: status: pending, key: test_scatter-d2ea151622695e6d528fc27bbb7e921d>,
 <Future: status: pending, key: test_scatter-cb2a3035a1594d48ff346cd16b966518>,
 <Future: status: pending, key: test_scatter-990d0309c449b19b33b32dea93757292>,
 <Future: status: pending, key: test_scatter-168027cba826239020ed323925faa983>]

In [36]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_broadcast-1cbbffb737592187644be238ee1f9682': ('tcp://10.2.171.62:42106',),
 'test_broadcast-72a1a64a7cc2cc322af24bb9d307b35d': ('tcp://10.2.171.62:37576',),
 'test_broadcast-75274ada5e6cad21e25e663bf5f43f73': ('tcp://10.2.171.62:34855',),
 'test_broadcast-bc708cdf85134882200b5f61b831f396': ('tcp://10.2.171.62:45457',),
 'test_point_to_point-1cb97e86c6e1dea824ab997523915c2c':

In [37]:
client.gather(nodes)

[True, True, True, True]

### Test Gather

In [38]:
def test_gather(r):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()

    sendbuf = np.zeros(100, dtype='i') + rank
    recvbuf = None
    if rank == 0:
        recvbuf = np.empty([size, 100], dtype='i')
    comm.Gather(sendbuf, recvbuf, root=0)
    if rank == 0:
        return [np.allclose(recvbuf[i,:], i) for i in range(size)]

Notice the order of scheduling matters here. If the receiver is created after the senders, a deadlock will occur. 

In [39]:
nodes = [client.submit(test_gather, future) for future in res]

In [40]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_broadcast-1cbbffb737592187644be238ee1f9682': ('tcp://10.2.171.62:42106',),
 'test_broadcast-72a1a64a7cc2cc322af24bb9d307b35d': ('tcp://10.2.171.62:37576',),
 'test_broadcast-75274ada5e6cad21e25e663bf5f43f73': ('tcp://10.2.171.62:34855',),
 'test_broadcast-bc708cdf85134882200b5f61b831f396': ('tcp://10.2.171.62:45457',),
 'test_gather-8599c37950f5c75361442b368ed038f1': ('tcp:/

In [41]:
client.gather(nodes)

[[True, True, True, True], None, None, None]

### Test AllGather
Note: This does not work yet. 

In [42]:
def test_all_gather(d):
    comm = MPI.COMM_WORLD
    p = comm.Get_size()
    xg = np.zeros(p, dtype='i')
    comm.Allgather([d[0,0],  MPI.INT],
                   [xg, MPI.INT])
    return xg.sum()

In [43]:
nodes = [client.submit(test_all_gather, future) for future in res]
nodes

[<Future: status: pending, key: test_all_gather-e80f88737d301e6f87b3201525cefded>,
 <Future: status: pending, key: test_all_gather-92f6219f562c034883f3d6a93176df79>,
 <Future: status: pending, key: test_all_gather-d585c8614048777355829e9cb6617e50>,
 <Future: status: pending, key: test_all_gather-4a14b909ef55da823143616a3d8623f4>]

In [44]:
client.who_has()

{'create_np-2fb2c90159778f8dde184a06e502f581': ('tcp://10.2.171.62:37576',),
 'create_np-46fbb7eeb4cad20b72102f33b0968993': ('tcp://10.2.171.62:45457',),
 'create_np-48ecd8dd8d9c7d3d4d428d1c8b05a8e5': ('tcp://10.2.171.62:34855',),
 'create_np-918cf2366aa5fa053ef38b999d0cbac8': ('tcp://10.2.171.62:42106',),
 'get_ranks-569e64d811f8ef6822088996082d3ea1': ('tcp://10.2.171.62:34855',),
 'get_ranks-8696fb4f9cd0ee5192b71c0e474898e4': ('tcp://10.2.171.62:42106',),
 'get_ranks-be89f0b062970fce534df6cb95cbcfa0': ('tcp://10.2.171.62:37576',),
 'get_ranks-c1c698d7265b3884daf93d8830905938': ('tcp://10.2.171.62:45457',),
 'test_all_gather-4a14b909ef55da823143616a3d8623f4': (),
 'test_all_gather-92f6219f562c034883f3d6a93176df79': (),
 'test_all_gather-d585c8614048777355829e9cb6617e50': (),
 'test_all_gather-e80f88737d301e6f87b3201525cefded': (),
 'test_broadcast-1cbbffb737592187644be238ee1f9682': ('tcp://10.2.171.62:42106',),
 'test_broadcast-72a1a64a7cc2cc322af24bb9d307b35d': ('tcp://10.2.171.62:37